In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
import os

In [2]:
data_dir = "./Big Star Collectibles"

In [3]:
files = os.listdir(data_dir)
file_texts = []
for file in files:
    with open(f"{data_dir}/{file}") as f:
        file_text = f.read()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=128, chunk_overlap=32,
    )
    texts = text_splitter.split_text(file_text)
    for i, chunked_text in enumerate(texts):
        file_texts.append(Document(page_content=chunked_text,metadata={ 
                    "doc_title": file.split(".")[0], 
                    "chunk_num": i}))

Created a chunk of size 139, which is longer than the specified 128
Created a chunk of size 151, which is longer than the specified 128
Created a chunk of size 151, which is longer than the specified 128
Created a chunk of size 139, which is longer than the specified 128
Created a chunk of size 130, which is longer than the specified 128
Created a chunk of size 188, which is longer than the specified 128
Created a chunk of size 130, which is longer than the specified 128


In [4]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

In [5]:
embeddings = HuggingFaceEmbeddings() # embed your data

/tmp/ipykernel_21387/3428753526.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings() # embed your data
/tmp/ipykernel_21387/3428753526.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings() # embed your data
/workspaces/advanced-rag-applications-with-vector-databases-3886256/ch1/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please upd

In [6]:
# store the embedded data into a vector database
vector_store = FAISS.from_documents(
    file_texts,
    embedding=embeddings
)

In [7]:
retriever = vector_store.as_retriever()

In [8]:
from dotenv import load_dotenv
# I added a .env file in the /workspaces/advanced-rag-applications-with-vector-databases-3886256/chapter_1 folder with the OPENAI_API_KEY value
# It is not saved in git because .gitignore has .env in it
# Also changes the instanciation of OpenAI below to point to Azure OpenAI endpoint

load_dotenv()

True

In [9]:
from langchain_openai import OpenAI

# added this to point to Azure OpenAI endpoint (see cell above for more info on .env file and OPENAI_API_KEY)
# Also added reference to 4o-mini model because I was getting 'Unknown model: gpt-3.5-turbo-instruct', perhaps
# that the model the older versions of the toolds refered in the req of this example are using by default
endpoint = "https://models.inference.ai.azure.com"
llm = OpenAI(base_url=endpoint, model="gpt-4o")

In [10]:
from langchain.prompts import ChatPromptTemplate
template="""You are a helpful assistant. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Cite your sources.
Question: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

print("Template: ",template)
print("Prompt: ",prompt)

Template:  You are a helpful assistant. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Cite your sources.
Question: {question} 
Context: {context} 
Answer:
Prompt:  input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are a helpful assistant. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nCite your sources.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# create a chain that combines the retriever, prompt, and llm
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
# Asked Cloud to generate this function to run the chain step by step with inspection points
# This function prints detailed information at each step, showing you exactly what's happening:
# Step 1: The initial question input
# Step 2: Retrieved context documents and the question dict
# Step 3: The formatted prompt after template substitution
# Step 4: The raw LLM output
# Step 5: The final parsed string output

def run_chain_with_inspection(question_input):
   
    print("=" * 60)
    print("STEP 1: Initial Input")
    print("=" * 60)
    print(f"Input: {question_input}")
    print()
    
    # Step 1: Create the dictionary with context and question
    print("=" * 60)
    print("STEP 2: Retrieve Context & Prepare Dict")
    print("=" * 60)
    
    # Retrieve relevant documents
    context_docs = retriever.invoke(question_input)
    print(f"Retrieved {len(context_docs)} documents")
    print(f"Context (first 200 chars): {str(context_docs)[:200]}...")
    
    # RunnablePassthrough() just passes the input through
    question = question_input
    print(f"Question: {question}")
    
    # Create the dictionary
    step1_output = {
        "context": context_docs,
        "question": question
    }
    print(f"\nStep 1 Output (dict keys): {step1_output.keys()}")
    print()
    
    # Step 2: Format with prompt template
    print("=" * 60)
    print("STEP 3: Format Prompt")
    print("=" * 60)
    step2_output = prompt.invoke(step1_output)
    print(f"Formatted Prompt Type: {type(step2_output)}")
    print(f"Formatted Prompt:\n{step2_output}")
    print()
    
    # Step 3: Send to LLM
    print("=" * 60)
    print("STEP 4: LLM Generation")
    print("=" * 60)
    step3_output = llm.invoke(step2_output)
    print(f"LLM Output Type: {type(step3_output)}")
    print(f"LLM Output : {step3_output}")
    print()
    
    # Step 4: Parse output to string
    print("=" * 60)
    print("STEP 5: Parse to String")
    print("=" * 60)
    parser = StrOutputParser()
    final_output = parser.invoke(step3_output)
    print(f"Final Output Type: {type(final_output)}")
    print(f"Final Output:\n{final_output}")
    print()
    
    print("=" * 60)
    print("CHAIN COMPLETE")
    print("=" * 60)
    
    return final_output

In [13]:
doitstepbystep = True
question = "When did Big Star Collectibles Launch? Cite where you found this information."

if doitstepbystep:
    print
    response = run_chain_with_inspection(question)
else:
    print ("Using the Langchain runnable chain")
    response = chain.invoke(question) # critical line

STEP 1: Initial Input
Input: When did Big Star Collectibles Launch? Cite where you found this information.

STEP 2: Retrieve Context & Prepare Dict
Retrieved 4 documents
Context (first 200 chars): [Document(metadata={'doc_title': 'Our Story', 'chunk_num': 1}, page_content='Launched officially in 2014, Saura was determined to create high-quality trading cards that were desirable and valuable for...
Question: When did Big Star Collectibles Launch? Cite where you found this information.

Step 1 Output (dict keys): dict_keys(['context', 'question'])

STEP 3: Format Prompt
Formatted Prompt Type: <class 'langchain_core.prompt_values.ChatPromptValue'>
Formatted Prompt:
messages=[HumanMessage(content="You are a helpful assistant. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nCite your sources.\nQuestion: When did Big Star Collectibles Launch? Cite where 

In [14]:
response

' Big Star Collectibles officially launched in 2014. [Our Story, chunk_num=1]'

The first answer was "' Big Star Collectibles officially launched in 2014. This information can be found in the "Our Story" document.<|fim_suffix|>'"

I resolved the problem by switching from gpt-4o-mini to gpt-4o, previously I tried another solution that did not work

Ignored solution proposed by Copilot:
You are getting "fims_suffix" (or similar placeholder text) instead of an actual document reference in cell 17 because the chain and prompt do not explicitly instruct the language model to include the metadata (such as document title or chunk number) from the retrieved documents in the context passed to the LLM.

By default, the retriever returns only the text content, not the metadata. If you want the LLM to cite sources, you need to:
1. Modify the prompt so it includes metadata (like `doc_title` and `chunk_num`) in the context.
2. Adjust how the context is constructed so that each chunk includes its source information.

To fix this, update the code that builds the context for the prompt so it formats each chunk with its metadata, for example:

def format_docs(docs):
    return "\n\n".join(
        f"Source: {doc.metadata.get('doc_title', 'unknown')} (chunk {doc.metadata.get('chunk_num', '?')}):\n{doc.page_content}"
        for doc in docs
    )

# Then, in your chain, use a RunnableLambda to format the context:
from langchain_core.runnables import RunnableLambda

chain = (
    {
        "context": retriever | RunnableLambda(format_docs),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

This way, the LLM will see the source information in the context and can cite it in its answer. If you want, I can update your notebook to implement this fix.

"fim_suffix" is not a standard output or variable from LangChain, OpenAI, or your notebook code. It is likely a hallucinated or placeholder string generated by the language model (LLM) when it does not have access to real citation metadata in the context.

This happens because:

The LLM is prompted to "Cite your sources," but the context it receives does not include explicit source information (like document titles or chunk numbers).
Without this information, the LLM may invent a placeholder like "fim_suffix" or similar, as it tries to fulfill the instruction to cite a source.
To get real citations, you must format the context passed to the LLM to include the actual metadata (such as document title and chunk number) for each chunk. Otherwise, the LLM will not know the true source and may generate made-up references.